In [1]:
import requests
import html


class question :
    def __init__(self,question: str,correct_answer: str,incorrect_answers: list):
        self.que = html.unescape(question)
        self.correct = html.unescape(correct_answer)
        self.incorrects = [html.unescape(i) for i in incorrect_answers]
        

def generate_que(parameters):
    response1 = requests.get(url="https://opentdb.com/api.php",params=parameters)
    data = response1.json()["results"]
    i = generate_list(data)
    return i

def generate_list(data):
    ques = []
    for i in data:
        q = question(i['question'],i['correct_answer'],i['incorrect_answers'])
        ques.append(q)
    return ques

In [3]:
import tkinter as tk
from random import shuffle
from tkinter import messagebox,PhotoImage

class MYGUI:
    
    def __init__(self):
        
        self.root = tk.Tk()
        
        width= self.root.winfo_screenwidth()
        height= self.root.winfo_screenheight()
        self.root.geometry("%dx%d" % (width, height))
        
        #change the file path of bg3.png after downloading
        bg = PhotoImage(file = "C:/Users/togat/simple quiz app/bg3.png")
        label1 = tk.Label(self.root, image = bg)
        label1.place(x = 0, y = 0)
        
        self.root.title("QUIZ APPLICATION")
        self.mframe = tk.Frame(self.root,bg="#fbf9e2")
        
        self.label1 = tk.Label(self.root,text="Quiz Time",font=('Arial',26),bg="#fbf9e2")
        self.label1.pack(pady=30)
        
        tk.Label(self.mframe,text="PLAY",font=('Arial',20),bg="#fbf9e2").pack(pady=10)
        
        #change the file path of play.png after downloading
        self.photo = PhotoImage(file="C:/Users/togat/simple quiz app/play.png")
        self.play = tk.Button(self.mframe,image=self.photo,borderwidth=0, 
                              bg="#fbf9e2",command=lambda: self.levels())
        
        self.play.pack(pady=20)
        
        self.mframe.pack()
        self.root.mainloop()


    def levels(self):
        
        self.mframe.forget()
        self.lframe = tk.Frame(self.root,bg="#fbf9e2")
        
        topic = self.droplist()
        
        list_of_cntry = ["easy","medium","hard"]  
        lv = tk.StringVar()
        
        drplist= tk.OptionMenu(self.lframe, lv, *list_of_cntry)  
        drplist.config(width=30)  
    
        lb1= tk.Label(self.lframe, text="Select Level", width=13,font=("arial",16),bg="#fbf9e2")
        
        lb1.pack()
        drplist.pack(pady=10)
        
        self.go = tk.Button(self.lframe,text="GO",font=('Arial',16),bg='black', 
                            fg="white", command=lambda: self.get_que(lv.get(),topic.get()))
        
        self.go.pack(pady=20)
        
        self.lframe.pack()
        

    def droplist(self):
        list_of_cntry = ["General Knowledge","Entertainment: Books","Entertainment: Music",
                         "Entertainment: Board Games","Entertainment: Cartoon & Animations",
                         "Science & Nature","Science: Mathematics","Science: Computers",
                         "Geography","Animals","Vehicles","Sports"]
        cv = tk.StringVar()
        
        drplist= tk.OptionMenu(self.lframe, cv, *list_of_cntry)  
        drplist.config(width=30)  
    
        lb2= tk.Label(self.lframe, text="Select Topic", width=13, font=("arial",16),bg="#fbf9e2")
        lb2.pack()
        
        drplist.pack(pady=10)
        return cv
    
    
    def get_que(self,level,topic):
        
        category = {"General Knowledge":9,"Entertainment: Books":10,"Entertainment: Music":12,
                    "Entertainment: Board Games":16,"Entertainment: Cartoon & Animations":32,
                    "Science & Nature":17,"Science: Mathematics":19,"Science: Computers":18,
                    "Geography":22,"Animals":27,"Vehicles":28,"Sports":21}

        print(level," ",topic)
        parameters = {"amount": 15,"type": "multiple", "category":category[topic],"difficulty":level}
        que = generate_que(parameters)
        self.quiz_display(que)
        
        
    def quiz_display(self,data):
        
        self.lframe.forget()
        
        self.user_answer = tk.StringVar()
        self.button_clicked = tk.BooleanVar()
        
        self.qno = self.score = 0
        self.qbank = data
        
        for i in data:
            self.eframe = tk.Frame(self.root,bg="#fbf9e2")
            self.qno += 1
            
            qstr = str(self.qno)+" / "+str(len(self.qbank))
            
            self.display_question(i.que)
            self.display_options(i.incorrects,i.correct)
            
            self.feedback = tk.Label(self.eframe, pady=10,font=("ariel", 15, "bold"),bg="#fbf9e2")
            self.feedback.pack()
            
            self.buttons()
            tk.Label(self.eframe,text=qstr,font=('Arial',16),bg="#fbf9e2").pack(pady=20,side="bottom")
            
            self.eframe.pack()
            self.eframe.wait_variable(self.button_clicked)
            self.eframe.forget()
        
        self.display_result()
        self.eframe.forget()
        self.mframe.pack()
        self.mframe.tkraise()


    def display_question(self,question):
        self.label2 = tk.Label(self.eframe,text=question,font=('Arial',16,"bold"),bg="#fbf9e2")
        self.label2.pack(padx=30,pady=20)
    
    
    def display_options(self,data1:list, data2:str):
        self.user_answer.set(None)
        choices = data1+[data2]
        shuffle(choices)
        R1 = {}
        for i in choices:
            R1[i] = tk.Radiobutton(self.eframe, text=i,variable=self.user_answer,
                                   value=i,font=("ariel", 14),bg="#fbf9e2",command=lambda: self.check_answer(data2,R1))
            R1[i].pack(pady=10)
        
    
    def check_answer(self,correct_answer,R1):
        for i in R1:
            R1[i]['state'] = 'disabled'
        user_answer = self.user_answer.get()
        if user_answer.lower() == correct_answer.lower():
            self.feedback["fg"] = "green"
            self.feedback["text"] = 'Correct answer! \U0001F44D'
            self.score += 1
        else:
            self.feedback['fg'] = 'red'
            self.feedback['text'] = ('\u274E Oops! \n'
                                     f'The right answer is: {correct_answer}')
    
    
    def buttons(self):
        next_button = tk.Button(self.eframe, text="Next", command=self.next_btn,
                             width=10, bg="green", fg="white", font=("ariel", 16, "bold"))
        
        next_button.pack()

        quit_button = tk.Button(self.eframe, text="Quit", command=self.root.destroy,
                             width=5, bg="red", fg="white", font=("ariel", 16, " bold"))

        quit_button.pack()
        
    
    def next_btn(self):
        if self.qno <= len(self.qbank):
            self.button_clicked.set(True)
    
    def display_result(self):
        wrong = self.qno - self.score
        score_percent = int(self.score / self.qno * 100)

        tk.messagebox.showinfo(title="result", 
                               message="result : {} %\nCorrect : {}\nWrong : {}"
                               .format(score_percent,self.score,wrong))
        

In [4]:
MYGUI()